In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from preprocessor import Preprocessor
from evaluator import ModelEvaluator
import pandas as pd
import numpy as np

In [2]:
'''
Удаляем мультиколлинерные признаки (vif > 5) и признаки с >30% пропусков
'''

'\nУдаляем мультиколлинерные признаки (vif > 5) и признаки с >30% пропусков\n'

In [3]:
df_train = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_train.csv")
df_test = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_test.csv")

In [4]:
def hypothesis_6(df):
    
    missing_threshold=0.3
    vif_threshold=5
    df = df.copy()
    
    missing_ratios = df.isnull().mean()
    cols_to_keep = missing_ratios[missing_ratios <= missing_threshold].index.tolist()
    df = df[cols_to_keep]
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    X = df[numeric_cols].dropna()
    
    vif_scores = {}
    for i, col in enumerate(numeric_cols):
        try:
            vif = variance_inflation_factor(X.values, i)
            vif_scores[col] = vif
        except:
            vif_scores[col] = np.inf
    
    low_vif_cols = [col for col, vif in vif_scores.items() if vif <= vif_threshold]

    non_numeric_cols = list(set(df.columns) - set(numeric_cols))
    final_cols = low_vif_cols + non_numeric_cols
    
    return df[final_cols]

In [5]:
preprocessor = Preprocessor(custom_steps=[hypothesis_6])

X_train_processed = preprocessor.transform(df_train.drop(columns=['TARGET', 'SK_ID_CURR']))
num_cols = X_train_processed.select_dtypes(include='number').columns
X_train_processed = X_train_processed[num_cols]


In [11]:
test_cols = X_train_processed.columns
X_test_processed = df_test[test_cols]
X_test_processed = X_test_processed.fillna(X_test_processed[test_cols].median())


In [12]:
evaluator = ModelEvaluator(LogisticRegression(max_iter=100))

roc_auc = evaluator.evaluate(X_train_processed, df_train['TARGET'])
print(f"ROC-AUC train: {roc_auc:.5f}")

evaluator.save_test_predictions(
    X_test=X_test_processed,
    ids=df_test['SK_ID_CURR'],
    filename='test_predictions_hyp_6.csv'
)
#ROC-AUC train: 0.55711

#ROC-AUC test: 0.56760
#ROC-AUC test baseline: 0.61204
#ROC-AUC test surplus: -0.04444


C:\Users\Роман\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ROC-AUC train: 0.55711
